# Denoising mel-spectrograms with a residual dense network

## Setup

### Environment

In [0]:
!pip install -q ipython_secrets gsheet-keyring comet_ml

     |████████████████████████████████| 204kB 4.9MB/s 
     |████████████████████████████████| 204kB 10.6MB/s 
     |████████████████████████████████| 419kB 15.9MB/s 
     |████████████████████████████████| 2.7MB 20.4MB/s 
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.


In [0]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
[[ -d ./apex ]] || git clone https://github.com/NVIDIA/apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
!sh setup.sh

setup.sh: 3: setup.sh: [[: not found
Cloning into 'apex'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 6665 (delta 41), reused 30 (delta 17), pack-reused 6595
Receiving objects: 100% (6665/6665), 13.74 MiB | 26.64 MiB/s, done.
Resolving deltas: 100% (4429/4429), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-8zu6uxt8
Created temporary directory: /tmp/pip-req-tracker-im7wgfzo
Created requirements tracker '/tmp/pip-req-tracker-im7wgfzo'
Created temporary directory: /tmp/pip-install-r_75y6h6
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-4h3uh8cr
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker

### Variables

In [0]:
from ipython_secrets import get_secret

CODEBASE = "https://github.com/sdll/audio-denoising"

PROJECT = "Arbeit"
COMET_ML_API_KEY = get_secret("comet-{}".format(PROJECT))

GDRIVE_MOUNT_POINT = "/content/drive"
TRAIN_DATASET = "dataset/train"
VAL_DATASET = "dataset/val"
WORKSPACE= "sdll"

from google.colab import drive
import os
drive.mount(GDRIVE_MOUNT_POINT)
GOOGLE_DRIVE_ROOT = GDRIVE_MOUNT_POINT + "/" + list(filter(lambda x: x[0] != '.', os.listdir(GDRIVE_MOUNT_POINT)))[0]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Data

In [0]:
!git clone $CODEBASE

Cloning into 'audio-denoising'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 51 (delta 20), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [0]:
%cd audio-denoising

/content/audio-denoising


In [0]:
!mkdir -p dataset

![[ -f ./dataset/train.zip ]] || wget https://www.dropbox.com/s/n6nhp5e231rl0b5/train.zip?raw=1 -O dataset/train.zip
![[ -f ./dataset/test.zip ]] || wget https://www.dropbox.com/s/nt4q2n0esiboc1i/val.zip?raw=1 -O dataset/val.zip

--2020-04-30 21:01:27--  https://www.dropbox.com/s/n6nhp5e231rl0b5/train.zip?raw=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/n6nhp5e231rl0b5/train.zip [following]
--2020-04-30 21:01:27--  https://www.dropbox.com/s/raw/n6nhp5e231rl0b5/train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc96efeb5c4b5c66185fa79264cd.dl.dropboxusercontent.com/cd/0/inline/A238IctzNkefDxx4Qurprg52xsufbLAzUAjFKRk5MErKHxitaXxXrWSnH_drn435FadbJFht8VqF40K9jGPkXfK45QCuNpWXIHBpZ40eUKPtxWtad_LHjhh9gF43SEuMkdk/file# [following]
--2020-04-30 21:01:27--  https://uc96efeb5c4b5c66185fa79264cd.dl.dropboxusercontent.com/cd/0/inline/A238IctzNkefDxx4Qurprg52xsufbLAzUAjFKRk5MErKHxitaXxXrWSnH_drn435FadbJFht8VqF40K9jGPkXfK45QCuNpWXIHBpZ40eUKPtxWtad_LHj

In [0]:
%cd dataset
![[ -d train ]] || unzip -qq train.zip
![[ -d val ]] || unzip -qq val.zip
%cd ..

/content/audio-denoising/dataset
/content/audio-denoising


In [0]:
![[ -d pytorch_ssim ]] || (git clone https://github.com/Po-Hsun-Su/pytorch-ssim && mv pytorch-ssim/pytorch_ssim . && rm -rf pytorch-ssim)

Cloning into 'pytorch-ssim'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


### Imports

In [0]:
%cd /content/audio-denoising/
from comet_ml import Experiment

import argparse
from timeit import default_timer as timer

import numpy as np
import seaborn as sns
import torch
from matplotlib import pyplot as plt
from torch import nn, optim
from tqdm import tqdm as tqdm_base

from apex import amp, optimizers
from audio_denoising.data.loader import load
from audio_denoising.model.rdn import ResidualDenseNetwork as Model
from pytorch_ssim import ssim


def tqdm(*args, **kwargs):
    if hasattr(tqdm_base, "_instances"):
        for instance in list(tqdm_base._instances):
            tqdm_base._decr_instances(instance)
    return tqdm_base(*args, **kwargs)

/content/audio-denoising


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
sns.set()

## Training

In [0]:
def get_arg_parser():
    parser = argparse.ArgumentParser()

    parser.add_argument("--batch-size", type=int, default=8)
    parser.add_argument("--growth-rate", type=int, default=24)
    parser.add_argument("--kernel-size", type=int, default=3)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--num-blocks", type=int, default=9)
    parser.add_argument("--num-channels", type=int, default=1)
    parser.add_argument("--num-epochs", type=int, default=80)
    parser.add_argument("--num-features", type=int, default=16)
    parser.add_argument("--num-layers", type=int, default=6)
    parser.add_argument("--seed", type=int, default=42)

    return parser


def get_criterion():
    return nn.MSELoss()


def get_optimizer(model, lr=1e-3):
    return optim.SGD(model.parameters(), lr)


def psnr(prediction, target, max_pixel=255.0):
    return 10.0 * ((max_pixel ** 2) / ((prediction - target) ** 2).mean()).log10()


def train(
    experiment,
    loader,
    model,
    criterion,
    optimizer,
    args,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    scheduler=None,
    use_amp=True,
    validator=None,  # (validate, val_loader)
    verbose=True,
):
    experiment.log_parameters(vars(args))

    np.random.seed(args.seed)

    # save the model weights with the best psnr
    if validator:
        best_psnr = 0.0

    for epoch in tqdm(range(args.num_epochs), desc="Epoch", unit="epochs"):
        with experiment.train():
            model.train()
            train_psnr = []
            train_ssim = []

            for clean_image, noisy_image in tqdm(
                loader, desc="Train images", unit="batches"
            ):
                image = noisy_image.to(device, dtype=torch.float)
                gt_image = clean_image.to(device, dtype=torch.float)
                noise = image - gt_image

                prediction = model(image)
                cleaned_image = image - prediction

                loss = criterion(prediction, noise)

                if use_amp:
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                else:
                    loss.backward()

                optimizer.step()
                optimizer.zero_grad()

                current_psnr = psnr(cleaned_image, gt_image).data.item()
                current_ssim = ssim(cleaned_image, gt_image).data.item()
                train_psnr.append(current_psnr)
                train_ssim.append(current_ssim)
                experiment.log_metric("psnr", current_psnr)
                experiment.log_metric("ssim", current_ssim)
                experiment.log_metric("loss", loss.data.item())

            experiment.log_metric("mean_psnr", np.mean(train_psnr))
            experiment.log_metric("mean_ssim", np.mean(train_ssim))
        if validator:
            validate, val_loader = validator
            test_psnr, _, _ = validate(experiment, val_loader, model, device, verbose)
            if scheduler:
                scheduler.step(test_psnr)
            if test_psnr > best_psnr:
                best_psnr = test_psnr
                filename_pth = GOOGLE_DRIVE_ROOT + '/audio_denoising_psnr_{:.4f}_epoch_{}.pth'.format(
                    test_psnr, epoch
                )
                torch.save(model.state_dict(), filename_pth)
    return model


def validate(
    experiment,
    loader,
    model,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    verbose=True,
):
    with experiment.test():
        model.eval()
        test_psnr = []
        test_ssim = []
        test_prediction_times = []
        for clean_image, noisy_image in tqdm(loader, desc="Val images", unit="batches"):
            image = noisy_image.to(device, dtype=torch.float)
            gt_image = clean_image.to(device, dtype=torch.float)
            start = timer()
            prediction = model(image)
            end = timer()
            cleaned_image = image - prediction

            prediction_time = end - start
            test_prediction_times.append(prediction_time)
            experiment.log_metric("prediction_time", prediction_time)

            current_psnr = psnr(cleaned_image, gt_image).data.item()
            current_ssim = ssim(cleaned_image, gt_image).data.item()

            test_psnr.append(current_psnr)
            test_ssim.append(current_ssim)

        test_psnr = np.mean(test_psnr)
        test_ssim = np.mean(test_ssim)
        test_prediction_time = np.mean(test_prediction_times)

        experiment.log_metric("mean_psnr", test_psnr)
        experiment.log_metric("mean_ssim", test_ssim)
        experiment.log_metric("mean_prediction_time", test_prediction_time)

    if verbose:
        print(
            "\nMean Test PSNR: {:.2f}\nMean Test SSIM: {:.2f}\nMean Prediction Time: {:.2f}".format(
                test_psnr, test_ssim, test_prediction_time
            )
        )
    return test_psnr, test_ssim, test_prediction_time

In [0]:
experiment = Experiment(
    api_key=COMET_ML_API_KEY,
    project_name=PROJECT,
    workspace=WORKSPACE,
    auto_output_logging=None,
)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/sdll/arbeit/02f057df8af24286bdfaae2af8794388



In [0]:
args = get_arg_parser().parse_args(args=[])

train_loader = load(TRAIN_DATASET, batch_size=args.batch_size)
val_loader = load(VAL_DATASET, batch_size=1)

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Model(args).to(device)
optimizer = get_optimizer(model, args.lr)
model, optimizer = amp.initialize(model, optimizer, opt_level="O3")
criterion = get_criterion()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=0, verbose=True)


Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


In [0]:
train(
    experiment,
    train_loader,
    model,
    criterion,
    optimizer,
    args,
    device,
    scheduler=scheduler,
    validator=(validate, val_loader)
)

Train images:   0%|          | 0/1500 [00:00<?, ?batches/s]


Mean Test PSNR: 59.63
Mean Test SSIM: 0.42
Mean Prediction Time: 0.01


Train images:   0%|          | 0/1500 [00:00<?, ?batches/s]


Mean Test PSNR: 59.71
Mean Test SSIM: 0.43
Mean Prediction Time: 0.01


Train images:   0%|          | 0/1500 [00:00<?, ?batches/s]

...


Mean Test PSNR: 59.71
Mean Test SSIM: 0.42
Mean Prediction Time: 0.01
Epoch    35: reducing learning rate of group 0 to 1.7858e-07.


Train images:   0%|          | 0/1500 [00:00<?, ?batches/s]


Mean Test PSNR: 59.71
Mean Test SSIM: 0.42
Mean Prediction Time: 0.01
Epoch    36: reducing learning rate of group 0 to 1.3394e-07.


Epoch:  45%|████▌     | 36/80 [6:20:19<7:44:50, 633.88s/epochs]


KeyboardInterrupt: ignored

In [0]:
filename_pth = GOOGLE_DRIVE_ROOT + '/audio_denoising_psnr_{:.4f}_epoch_{}.pth'.format(
    59.71, 35 
)
torch.save(model.state_dict(), filename_pth)

## Validation

In [0]:
ckpt = GOOGLE_DRIVE_ROOT + '/audio_denoising_psnr_59.7472_epoch_6.pth'
model.load_state_dict(torch.load(ckpt))

<All keys matched successfully>

In [0]:
test_psnr, test_ssim, test_prediction_time = validate(experiment, val_loader, model, verbose=True)


Val images: 100%|██████████| 2000/2000 [00:54<00:00, 36.89batches/s]


Mean Test PSNR: 59.75
Mean Test SSIM: 0.43
Mean Prediction Time: 0.01


In [16]:
%matplotlib inline
from IPython.display import clear_output, display
import sys
import time
for clean_mel, noisy_mel in val_loader: 
    time.sleep(.25)
    clean_mel = clean_mel.to("cpu", dtype=torch.float)
    noisy_mel = noisy_mel.to("cpu", dtype=torch.float)
    fig = plt.figure(figsize=(17, 8), dpi=100)
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    ax1.imshow(clean_mel.squeeze(), interpolation='nearest', aspect='auto')
    ax1.set_title('Clean image')
    ax2.imshow((noisy_mel - model(noisy_mel.to(device)).data.to("cpu")).squeeze(),
               interpolation='nearest', aspect='auto')
    ax2.set_title('Cleaned image')
    display(fig)
    clear_output(wait=True)
    sys.stdout.flush()


Buffered data was truncated after reaching the output size limit.

## Scratchpad

In [0]:
dataset = load(TRAIN_DATASET, batch_size=1)
heights = []
for clean_mel, noisy_mel in dataset:
    heights.append(clean_mel.shape[2]) 

In [0]:
plt.figure(dpi=120)
plt.title("Distribution of Train MEL Heights")
plt.xlabel("Height, px")
plt.ylabel("Frequency")
sns.distplot(heights)
plt.show()

In [0]:
min(heights), max(heights)

In [0]:
%cd /content/audio-denoising/
!git pull

In [0]:
%%capture
args = get_arg_parser().parse_args(args=[])

train_loader = load(TRAIN_DATASET, batch_size=args.batch_size)
val_loader = load(VAL_DATASET, batch_size=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

experiment = Experiment(
    api_key=COMET_ML_API_KEY,
    project_name=PROJECT,
    workspace=WORKSPACE,
    auto_output_logging=None,
)

grid_search = {
    "num_blocks": np.arange(3, 12, 3),
    "num_layers": np.arange(6, 18, 3),
    "num_features": np.arange(8, 32, 8),
    "growth_rate": np.arange(8, 32, 8),
}

best = {
    "num_blocks": None,
    "num_layers": None,
    "num_features": None,
    "growth_rate": None,
}

args.num_epochs = 2

best_psnr = 0.0
for num_blocks in grid_search["num_blocks"]:
    for num_layers in grid_search["num_layers"]:
        for num_features in grid_search["num_features"]:
            for growth_rate in grid_search["growth_rate"]:
                setattr(args, "num_blocks", num_blocks)
                setattr(args, "num_layers", num_layers)
                setattr(args, "num_features", num_features)
                setattr(args, "growth_rate", growth_rate)
                model = Model(args).to(device)
                optimizer = get_optimizer(model, args.lr)
                model, optimizer = amp.initialize(model, optimizer, opt_level="O3")
                criterion = get_criterion()
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                    optimizer, mode="max", factor=0.75, patience=0, verbose=True
                )
                train(
                    experiment,
                    [next(iter(train_loader))],
                    model,
                    criterion,
                    optimizer,
                    args,
                    device,
                    scheduler=scheduler,
                )
                test_psnr, _, _ = validate(
                    experiment, [next(iter(val_loader))], model, device, verbose=False
                )
                if test_psnr > best_psnr:
                    best_psnr = test_psnr
                    best["num_blocks"] = num_blocks
                    best["num_layers"] = num_layers
                    best["num_features"] = num_features
                    best["growth_rate"] = growth_rate
                del model, optimizer, criterion, scheduler
                torch.cuda.empty_cache()
experiment.end()


COMET INFO: Experiment is live on comet.ml https://www.comet.ml/sdll/arbeit/170b25823ed849329d44c149068dafe9

Train images:   0%|          | 0/1 [00:00<?, ?batches/s]

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


Train images:   0%|          | 0/1 [00:00<?, ?batches/s]

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


Train images:   0%|          | 0/1 [00:00<?, ?batches/s]

...

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


Train images:   0%|          | 0/1 [00:00<?, ?batches/s]

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


Epoch:   0%|          | 0/2 [00:00<?, ?epochs/s]


RuntimeError: ignored

In [0]:
print(best)

{'num_blocks': 9, 'num_layers': 6, 'num_features': 16, 'growth_rate': 24}
